In [3]:
import numpy as np
import pcabm.dcbm as dc
import pcabm.pcabm as pca
import pcabm.plem as plem 
import pcabm.sc as sc
import pcabm.commFunc as cf
from sklearn.metrics.cluster import adjusted_rand_score

# Data Generation
This is the simulation in Section 6.1. 

In [4]:
## Parameter Settings
p, n, k = 5,200,2
rho = 2*np.log(n)**1.5/n                  # sparsity level
gamma = np.arange(0.4,2.1,0.4)            # covariate coefficients

## Generate Covariates
Z = np.zeros((p,n,n))
Z[0,:,:] = np.random.binomial(1,0.1,size=(n,n))
Z[1,:,:] = np.random.poisson(0.1,size=(n,n))
Z[2,:,:] = np.random.uniform(0,1,size=(n,n))
Z[3,:,:] = np.random.exponential(0.3,size=(n,n))
Z[4,:,:] = np.random.normal(0,0.3,size=(n,n))
Z = np.tril(Z,-1)
Z = Z+np.transpose(Z,axes=(0,2,1))
Z = np.transpose(Z,axes=(1,2,0))

# Generate Edge
B = np.ones((n,n));B[0:(n//2),0:(n//2)]=2;B[(n//2):n,(n//2):n]=2;B=B*rho
lam = np.multiply(B,np.exp(np.dot(Z,gamma)))
A = np.random.poisson(np.tril(lam,-1)); Ab = A+A.T
gt=np.array([0]*(n//2)+[1]*(n//2))      # ground truth

# PCABM.SCWA
$\gamma$ estimation and SCWA

In [5]:
modelSCWA,gamma_est = sc.SCWA_r(Ab,Z,2)
print('ARI is', adjusted_rand_score(modelSCWA.labels_,gt))

ARI is 1.0


Confidence interval for $\gamma$

In [6]:
FI = cf.Info(modelSCWA.labels_,gamma_est,Ab,Z)
print('Lower: ',gamma_est-1.96*np.sqrt(np.diagonal(np.linalg.inv(FI))))
print('Upper: ',gamma_est+1.96*np.sqrt(np.diagonal(np.linalg.inv(FI))))

Lower:  [0.38745402 0.74939339 1.16190236 1.55999874 1.92752696]
Upper:  [0.46922884 0.81388913 1.26375516 1.60951883 2.02149452]


# PCABM.PLEM0
We directly apply the $\hat{\gamma}$ estimated in the last step. This may give a bad result. Usually, we need to try multiple initializations.

In [10]:
modelPCA = plem.PLEM(Ab,Z,2,gamma_est)
estPCA,_ = modelPCA.fit(community_init=np.random.randint(2, size=n),gt=gt)
print('ARI is', adjusted_rand_score(estPCA,gt))

ARI is 1.0


#  PCABM.PLEM
We use the label estimated from SCWA as the initialization of tabu search, which usually results in more stable results.

In [11]:
modelPCA = plem.PLEM(Ab,Z,2,gamma_est)
estPCA,_ = modelPCA.fit(community_init=modelSCWA.labels_,gt=gt)
print('ARI is', adjusted_rand_score(estPCA,gt))

ARI is 1.0


# SBM.SC_r

In [9]:
_,_,modelSC = sc.SC_r(Ab,2)
print('ARI is', adjusted_rand_score(modelSC.labels_,gt))

ARI is -0.00019903458175581754


# SCORE

In [7]:
modelSCORE = sc.SCORE(Ab,2)
print('ARI is', adjusted_rand_score(modelSCORE.labels_,gt))

ARI is 0.0
